In [12]:
%load_ext autoreload
%autoreload 2

# %set_env CUDA_VISIBLE_DEVICES=7
# import sys; sys.path.append('/future/u/okhattab/repos/public/stanfordnlp/dspy')

import dspy
from dspy.predict import Predict
import dsp
from dsp.primitives.vision import Image
from dsp.modules.gpt4vision import GPT4Vision

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 1) Configure the VLM

In [13]:
gpt = GPT4Vision()
dspy.settings.configure(lm=gpt)

### 2) Ask questions about the scene

In [17]:
#Define a simple signature for basic question answering
class RobotBrain(dspy.Signature):
    """Imagine you have a robot that can move in 3D space and has a gripper."""
    task:str = dspy.InputField()
    image = dspy.InputField()
    actions:list[dict] = dspy.OutputField(desc="[{x=?, y=?, z=?, roll=?, pitch=?, yaw=?, open_or_close_gripper=?}]")

gpt = GPT4Vision()
dspy.settings.configure(lm=gpt, num_generations=1)
#Pass signature to ChainOfThought module
generate_answer = dspy.Predict(RobotBrain)

# Call the predictor on a particular input alongside a hint.
task='What are the next 3 action should the robot take to pick up the block?'
image= Image('gripper_mid_to_grasp.png')
hint = "It should be a list of gripper poses and graps. Usually, the depth is underestimated."
pred = generate_answer(task=task, image=image, hint=hint)


print(f"Question: {task}")
print(f"Predicted Answer: {pred.actions}")
gpt.inspect_history(n=10)
image.pil.show()

PydanticUserError: Field 'image' requires a type annotation

For further information visit https://errors.pydantic.dev/2.6/u/model-field-missing-annotation